# VacationPy
---

## Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/valkeeranan/Desktop/python-api-challenge/WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kavaratti,10.5669,72.6420,27.37,76,15,3.02,IN,1730854275
1,1,prince rupert,54.3161,-130.3201,8.83,90,100,19.03,CA,1730854276
2,2,young,-32.6833,-57.6333,14.60,65,2,3.87,UY,1730854277
3,3,edinburgh of the seven seas,-37.0676,-12.3116,14.47,82,99,3.91,SH,1730854278
4,4,wailua homesteads,22.0669,-159.3780,26.44,89,0,2.06,US,1730854279


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size=20,
    color="City",
    cmap="Category20",
    hover_cols=["City", "Humidity"],
    title="City Humidity Map",
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head(20)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,wailua homesteads,22.0669,-159.3780,26.44,89,0,2.06,US,1730854279
26,26,wamba,8.9333,8.6000,22.03,42,0,1.60,NG,1730854305
92,92,isla mujeres,21.2311,-86.7310,26.03,85,0,2.87,MX,1730854381
97,97,natchez,31.5604,-91.4032,22.69,94,0,3.60,US,1730854386
103,103,lihue,21.9789,-159.3672,26.77,86,0,2.06,US,1730854029
116,116,san patricio,28.0170,-97.5169,21.97,58,0,2.82,US,1730854407
137,137,gunjur,13.2019,-16.7339,26.04,100,0,2.06,GM,1730854433
153,153,bonthe,7.5264,-12.5050,25.66,86,0,1.70,SL,1730854452
176,176,cabo san lucas,22.8909,-109.9124,22.10,68,0,3.91,MX,1730854477
186,186,remire-montjoly,4.9167,-52.2667,26.02,89,0,0.51,GF,1730854490


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,kavaratti,IN,10.5669,72.6420,76,
1,prince rupert,CA,54.3161,-130.3201,90,
2,young,UY,-32.6833,-57.6333,65,
3,edinburgh of the seven seas,SH,-37.0676,-12.3116,82,
4,wailua homesteads,US,22.0669,-159.3780,89,
5,tranby,NO,59.8167,10.2500,96,
6,stanley,GB,54.8680,-1.6985,93,
7,punta arenas,CL,-53.1500,-70.9167,70,
8,waitangi,NZ,-43.9535,-176.5597,89,
9,puerto deseado,AR,-47.7503,-65.8938,28,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(20)

Starting hotel search
kavaratti - nearest hotel: No hotel found
prince rupert - nearest hotel: Crest Hotel
young - nearest hotel: Hotel El Galeon
edinburgh of the seven seas - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
tranby - nearest hotel: Origo Leilighetshotell
stanley - nearest hotel: Hotel 52
punta arenas - nearest hotel: Hotel Lacolet
waitangi - nearest hotel: Hotel Chathams
puerto deseado - nearest hotel: Los Olmos
port saint john's - nearest hotel: Outback Inn
santa vitoria do palmar - nearest hotel: Hotel Brasil
tazovsky - nearest hotel: Тазовчанка
tobelo - nearest hotel: JJ Guesthouse
unye - nearest hotel: No hotel found
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
weno - nearest hotel: High Tide Hotel
olonkinbyen - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
mahina - nearest hotel: Motu Martin
rurrenabaque - nearest hotel: Hostal Madidi
enewe

,City,Country,Lat,Lng,Humidity,Hotel Name
0,kavaratti,IN,10.5669,72.6420,76,No hotel found
1,prince rupert,CA,54.3161,-130.3201,90,Crest Hotel
2,young,UY,-32.6833,-57.6333,65,Hotel El Galeon
3,edinburgh of the seven seas,SH,-37.0676,-12.3116,82,No hotel found
4,wailua homesteads,US,22.0669,-159.3780,89,Hilton Garden Inn Kauai Wailua Bay
5,tranby,NO,59.8167,10.2500,96,Origo Leilighetshotell
6,stanley,GB,54.8680,-1.6985,93,Hotel 52
7,punta arenas,CL,-53.1500,-70.9167,70,Hotel Lacolet
8,waitangi,NZ,-43.9535,-176.5597,89,Hotel Chathams
9,puerto deseado,AR,-47.7503,-65.8938,28,Los Olmos


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size=20,
    color="City",
    cmap="Category20",
    hover_cols=["Hotel Name", "Country"],
    title="Hotel Locations Map",
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)